In [1]:

from keras.utils.generic_utils import get_custom_objects 
import math, re, os
import random
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import layers
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from kaggle_datasets import KaggleDatasets
from tensorflow import keras
from functools import partial
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
print("Tensorflow version " + tf.__version__)
from sklearn.metrics import accuracy_score
from collections import Counter
from tensorflow.keras.preprocessing.image import ImageDataGenerator



Tensorflow version 2.3.1


In [2]:
def decode_image(path, label=None, target_size=(512, 512)):
    img = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    img = tf.cast(img, tf.float32) / 255.0
    img = tf.image.resize(img, target_size)
    
    return img if label is None else img, label

def data_augment(img, label=None):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    
    return img if label is None else img, label

class FixedDropout(layers.Dropout):
    def _get_noise_shape(self, inputs):
        if self.noise_shape is None:
            return self.noise_shape
 
        symbolic_shape = K.shape(inputs)
        noise_shape = [symbolic_shape[axis] if shape is None else shape
                       for axis, shape in enumerate(self.noise_shape)]
        return tuple(noise_shape)
strategy = tf.distribute.get_strategy()
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 16

load_dir = "/kaggle/input/cassava-leaf-disease-classification/"
sub_df = pd.read_csv(load_dir + 'sample_submission.csv')
sub_df['paths'] = load_dir + "/test_images/" + sub_df.image_id




get_custom_objects()['FixedDropout'] = FixedDropout
preds_all = []
models = []
preds_model = []

# ds_test = generator(JPEG_PATH_TR, tr.image_id.values)
for fold in range(5):
    print(f"## FOLD: {fold}")
    # BUILD MODEL
    K.clear_session()
    
    models.append(tf.keras.models.load_model(
        "/kaggle/input/notebook4edda1091f/EfficientNetB7-fold-%i.h5"%fold)
         )
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(sub_df.paths.values)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE).prefetch(AUTO))
preds = models[0].predict(test_dataset, verbose=1)
for i in range(1,5):
    test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(sub_df.paths.values)
    .map(decode_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE).prefetch(AUTO))
    preds += models[0].predict(test_dataset, verbose=1)


sub_df['label'] = preds.argmax(axis=1)
sub_df.drop(columns='paths').to_csv('submission.csv', index=False)


## FOLD: 0
## FOLD: 1
## FOLD: 2
## FOLD: 3
## FOLD: 4
1/1 [==============================] - 0s 29ms/step
